In [1]:
import pandas as pd
import numpy as np
import requests
from yahooquery import Ticker  # Step 2 (Speed)
import yfinance as yf          # Step 3 (Reliability)
import io
import json
import os
import time


In [2]:
import os
import json

# ==========================================
# CONFIGURATION
# ==========================================
# 1. FOLDER SETUP (The Fix for GitHub Portability)
DATA_FOLDER = "YfinanceDataDump"  # Relative path (creates folder in project root)

# Create the folder if it doesn't exist
if not os.path.exists(DATA_FOLDER):
    try:
        os.makedirs(DATA_FOLDER)
        print(f"Created data folder: {DATA_FOLDER}")
    except Exception as e:
        print(f"Warning: Could not create folder '{DATA_FOLDER}'. Saving to current directory. Error: {e}")
        DATA_FOLDER = "."

# 2. FILE PATHS (Everything saves inside the folder now)
CACHE_FILE = os.path.join(DATA_FOLDER, "financial_cache.json")
FORTRESS_CSV = os.path.join(DATA_FOLDER, "fortress_stocks.csv")
STRONG_CSV = os.path.join(DATA_FOLDER, "strong_stocks.csv")
RISKY_CSV = os.path.join(DATA_FOLDER, "risky_stocks.csv")
ANALYST_CSV = os.path.join(DATA_FOLDER, "Analyst_Fortress_Picks.csv")
BUFFETT_CSV = os.path.join(DATA_FOLDER, "Buffett_Value_Picks.csv")

# 3. UNIVERSE FILTERS
MIN_PRICE = 2.00               
MIN_VOLUME = 1_000_000       
MIN_CAP = 300_000_000        # $300M
MIN_CURRENT_RATIO = 1.2
MAX_PE_RATIO = 100.0         

# 4. SAFETY THRESHOLDS 
MIN_INTEREST_COVERAGE = 1.5
MIN_ROIC = 0.05              # 5%
FORTRESS_MARGIN_THRESHOLD = 0.05  # 5%

EXCLUDED_SECTORS = ['Financial Services', 'Real Estate']
CACHE_EXPIRY_DAYS = 30 

# ==========================================
# HELPER FUNCTIONS
# ==========================================
def load_cache():
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE, 'r') as f:
                return json.load(f)
        except:
            return {}
    return {}

def save_cache(cache_data):
    try:
        with open(CACHE_FILE, 'w') as f:
            json.dump(cache_data, f)
    except Exception as e:
        print(f"Warning: Could not save cache: {e}")

def calculate_altman_z_yfinance(bs, fin, market_cap):
    """
    Calculates Z-Score using yfinance DataFrames.
    Formula: 1.2A + 1.4B + 3.3C + 0.6D + 1.0E
    """
    try:
        # Helper to safely get value from Series
        def get_val(df, keys):
            for k in keys:
                if k in df.index:
                    return df.loc[k].iloc[0]
            return 0

        # Map yfinance row names
        total_assets = get_val(bs, ['Total Assets'])
        total_liab = get_val(bs, ['Total Liabilities Net Minority Interest', 'Total Liabilities'])
        current_assets = get_val(bs, ['Current Assets', 'Total Current Assets'])
        current_liab = get_val(bs, ['Current Liabilities', 'Total Current Liabilities'])
        retained_earnings = get_val(bs, ['Retained Earnings'])
        
        ebit = get_val(fin, ['EBIT', 'Operating Income'])
        total_revenue = get_val(fin, ['Total Revenue'])
        
        if total_assets == 0 or total_liab == 0: return 0

        # A: Working Capital / Total Assets
        A = (current_assets - current_liab) / total_assets
        
        # B: Retained Earnings / Total Assets
        B = retained_earnings / total_assets
        
        # C: EBIT / Total Assets
        C = ebit / total_assets
        
        # D: Market Value of Equity / Total Liabilities
        D = market_cap / total_liab
        
        # E: Sales / Total Assets
        E = total_revenue / total_assets
        
        return (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
    except Exception as e:
        return 0

In [3]:
# ==========================================
# STEP 1: FETCH COMBINED UNIVERSE (USA + CAD)
# ==========================================
def get_combined_universe():
    print("--- STEP 1: Fetching North American Universe ---")
    tickers = []
    
    # 1. USA
    try:
        url_us = "https://www.nasdaqtrader.com/dynamic/symdir/nasdaqtraded.txt"
        s = requests.get(url_us).content
        df_us = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='|')
        df_us = df_us[(df_us['Test Issue'] == 'N') & (df_us['ETF'] == 'N')]
        us_list = [x.replace('$', '-') for x in df_us['Symbol'].astype(str) if len(x) < 5]
        tickers.extend(us_list)
        print(f"   -> Found {len(us_list)} US stocks.")
    except:
        print("   -> Error fetching USA list.")

    return tickers

In [4]:
# ==========================================
# STEP 2: LIGHTWEIGHT SIEVE (YahooQuery)
# ==========================================
def get_initial_survivors(tickers):
    print(f"\n--- STEP 2: Running 'Lightweight' Filter on {len(tickers)} stocks ---")
    chunk_size = 500 
    survivors = []
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]
    
    for i, chunk in enumerate(chunks):
        if i % 2 == 0: print(f" -> Processing Batch {i+1}/{len(chunks)}...")
        try:
            yq = Ticker(chunk, asynchronous=True)
            df_modules = yq.get_modules('summaryProfile summaryDetail financialData price defaultKeyStatistics')
            
            for symbol, data in df_modules.items():
                if isinstance(data, str): continue 
                try:
                    price = data.get('price', {}).get('regularMarketPrice', 0)
                    if price is None: price = 0
                    
                    vol = data.get('summaryDetail', {}).get('averageVolume', 0)
                    if vol is None or vol == 0:
                         vol = data.get('price', {}).get('averageDailyVolume10Day', 0)
                    
                    cap = data.get('price', {}).get('marketCap', 0)
                    if cap is None: cap = 0
                    
                    sector = data.get('summaryProfile', {}).get('sector', 'Unknown')
                    fin_data = data.get('financialData', {})
                    curr_ratio = fin_data.get('currentRatio', 0)
                    op_margins = fin_data.get('operatingMargins', 0)
                    if curr_ratio is None: curr_ratio = 0
                    if op_margins is None: op_margins = 0

                    # --- P/E RATIO CHECK ---
                    pe = data.get('summaryDetail', {}).get('trailingPE')
                    if pe is not None and pe > MAX_PE_RATIO: continue

                    # FILTERS
                    if price < MIN_PRICE: continue
                    if cap < MIN_CAP: continue
                    if vol < MIN_VOLUME: continue 
                    if any(x in sector for x in EXCLUDED_SECTORS): continue
                    if curr_ratio < MIN_CURRENT_RATIO: continue
                    if op_margins <= 0: continue 
                    
                    survivors.append({
                        'Ticker': symbol,
                        'Sector': sector,
                        'Price': price,
                        'Op Margin %': round(op_margins * 100, 2),
                        'P/E': round(pe, 2) if pe else 0,
                        'Curr Ratio': curr_ratio,
                        'Mkt Cap (B)': round(cap / 1_000_000_000, 2)
                    })
                except: continue
        except: continue
    return pd.DataFrame(survivors)

In [5]:


# ==========================================
# STEP 3: DEEP DIVE (yfinance + Cache)
# ==========================================
def get_advanced_metrics(survivor_df):
    tickers = survivor_df['Ticker'].tolist()
    print(f"\n--- STEP 3: Fetching Deep Financials for {len(tickers)} Survivors ---")
    
    cache = load_cache()
    current_time = time.time()
    expiry_seconds = CACHE_EXPIRY_DAYS * 86400
    
    final_data = []
    
    for i, ticker in enumerate(tickers):
        if i % 20 == 0: print(f" -> Analyzing {i+1}/{len(tickers)}: {ticker}...")
        
        def determine_tier(metrics, base_row):
            if metrics['int_cov'] < MIN_INTEREST_COVERAGE: return "Risky"
            if metrics['roic'] < MIN_ROIC: return "Risky"
            
            op_margin_pct = base_row['Op Margin %'] / 100
            if op_margin_pct >= FORTRESS_MARGIN_THRESHOLD: return "Fortress"
            return "Strong"

        # 1. CHECK CACHE
        cached_data = cache.get(ticker)
        if cached_data and (current_time - cached_data['timestamp'] < expiry_seconds):
            if cached_data.get('roic') == -999: continue 
            
            base_row = survivor_df[survivor_df['Ticker'] == ticker].iloc[0]
            tier = determine_tier(cached_data, base_row)
            
            final_data.append({
                'Ticker': ticker,
                'Tier': tier,
                'Price': base_row['Price'],
                'P/E': base_row['P/E'],
                'Sector': base_row['Sector'],
                'Z-Score': cached_data['z_score'],
                'ROIC %': round(cached_data['roic'] * 100, 2),
                'Op Margin %': base_row['Op Margin %'],
                'Curr Ratio': base_row['Curr Ratio'],
                'Int Cov': cached_data['int_cov'],
                'Mkt Cap (B)': base_row['Mkt Cap (B)']
            })
            continue

        # 2. FETCH NEW DATA
        try:
            stock = yf.Ticker(ticker)
            fin = stock.financials
            bs = stock.balance_sheet
            
            if fin.empty or bs.empty:
                cache[ticker] = {'timestamp': current_time, 'z_score': 0, 'roic': -999, 'int_cov': -999}
                continue
            
            def get_item(df, keys):
                for k in keys:
                    if k in df.index: return df.loc[k].iloc[0]
                return 0

            ebit = get_item(fin, ['EBIT', 'Operating Income', 'Pretax Income'])
            int_exp = get_item(fin, ['Interest Expense', 'Interest Expense Non Operating'])
            total_assets = get_item(bs, ['Total Assets'])
            curr_liab = get_item(bs, ['Current Liabilities', 'Total Current Liabilities'])
            
            int_exp = abs(int_exp)
            if int_exp == 0: int_cov = 100
            else: int_cov = ebit / int_exp
            
            invested_cap = total_assets - curr_liab
            if invested_cap <= 0: roic = 0
            else: roic = ebit / invested_cap
            
            base_row = survivor_df[survivor_df['Ticker'] == ticker].iloc[0]
            mkt_cap_raw = base_row['Mkt Cap (B)'] * 1_000_000_000
            z = calculate_altman_z_yfinance(bs, fin, mkt_cap_raw)
            
            metrics = {
                'timestamp': current_time,
                'z_score': round(z, 2),
                'roic': roic,
                'int_cov': round(int_cov, 2)
            }
            cache[ticker] = metrics
            
            tier = determine_tier(metrics, base_row)

            final_data.append({
                'Ticker': ticker,
                'Tier': tier,
                'Price': base_row['Price'],
                'P/E': base_row['P/E'],
                'Sector': base_row['Sector'],
                'Z-Score': round(z, 2),
                'ROIC %': round(roic * 100, 2),
                'Op Margin %': base_row['Op Margin %'],
                'Curr Ratio': base_row['Curr Ratio'],
                'Int Cov': round(int_cov, 2),
                'Mkt Cap (B)': base_row['Mkt Cap (B)']
            })
            
        except Exception as e:
            continue

    save_cache(cache)
    return pd.DataFrame(final_data)

In [6]:
# ==========================================
# MAIN EXECUTION
# ==========================================
if __name__ == "__main__":
    tickers = get_combined_universe()
    
    if len(tickers) > 0:
        survivors_df = get_initial_survivors(tickers)
        
        if not survivors_df.empty:
            print(f"\n✅ Step 2 Complete. {len(survivors_df)} stocks passed basic filters.")
            
            final_results = get_advanced_metrics(survivors_df)
            
            if not final_results.empty:
                final_results = final_results.sort_values(by=['Tier', 'Z-Score'], ascending=[True, False])
                
                # 1. Standard Split
                fortress_df = final_results[final_results['Tier'] == 'Fortress'].copy()
                strong_df = final_results[final_results['Tier'] == 'Strong'].copy()
                risky_df = final_results[final_results['Tier'] == 'Risky'].copy()
                
                # 3. Save Files (Updated to use Relative Paths from Cell 2)
                try:
                    fortress_df.to_csv(FORTRESS_CSV, index=False)
                    strong_df.to_csv(STRONG_CSV, index=False)
                    risky_df.to_csv(RISKY_CSV, index=False)
                    
                    print("\n" + "="*60)
                    print("RESULTS GENERATED")
                    print("="*60)
                    print(f"1. FORTRESS ({len(fortress_df)}): Saved to '{FORTRESS_CSV}'")
                    print(f"2. STRONG   ({len(strong_df)}): Saved to '{STRONG_CSV}'")
                    print(f"3. RISKY    ({len(risky_df)}): Saved to '{RISKY_CSV}'")
                except Exception as e:
                    print(f"\n⚠️ Error Saving Files: {e}")
                    print("Check if the file is open in Excel or if the folder exists.")
                
                pd.set_option('display.max_rows', 500)
                pd.set_option('display.max_columns', 20)
                pd.set_option('display.width', 1000)
                
                print("\n--- FORTRESS PREVIEW ---")
                print(fortress_df.head(15))
            else:
                print("No stocks passed the deep financial analysis.")
        else:
            print("No stocks passed the initial lightweight filter.")
    else:
        print("Could not fetch ticker universe.")

--- STEP 1: Fetching North American Universe ---
   -> Found 6014 US stocks.

--- STEP 2: Running 'Lightweight' Filter on 6014 stocks ---
 -> Processing Batch 1/13...
 -> Processing Batch 3/13...
 -> Processing Batch 5/13...
 -> Processing Batch 7/13...
 -> Processing Batch 9/13...
 -> Processing Batch 11/13...
 -> Processing Batch 13/13...

✅ Step 2 Complete. 554 stocks passed basic filters.

--- STEP 3: Fetching Deep Financials for 554 Survivors ---
 -> Analyzing 1/554: A...
 -> Analyzing 21/554: ALGM...
 -> Analyzing 41/554: AQN...
 -> Analyzing 61/554: AXTA...
 -> Analyzing 81/554: BMBL...
 -> Analyzing 101/554: CDE...
 -> Analyzing 121/554: COP...
 -> Analyzing 141/554: CXW...
 -> Analyzing 161/554: DT...
 -> Analyzing 181/554: ETN...
 -> Analyzing 201/554: FOLD...
 -> Analyzing 221/554: GILD...
 -> Analyzing 241/554: HAFN...
 -> Analyzing 261/554: HUM...
 -> Analyzing 281/554: ISRG...
 -> Analyzing 301/554: LEN...
 -> Analyzing 321/554: MDT...
 -> Analyzing 341/554: NAGE...
 -> A

In [7]:
# 1. Define the function (if you haven't already in a previous cell)
def get_analyst_fortress_from_var(df_input):
    working_df = df_input.copy()
    tickers = working_df['Ticker'].tolist()
    
    print(f"\n--- STEP 4: Fetching Analyst Ratings for {len(tickers)} Stocks (From Memory) ---")
    print("    (Fetching serially to avoid throttling...)")
    
    analyst_data = []
    
    for i, ticker in enumerate(tickers):
        if i % 10 == 0: print(f" -> Analyst Scan {i+1}/{len(tickers)}: {ticker}...")
        
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            rec_mean = info.get('recommendationMean')
            target_price = info.get('targetMeanPrice')
            current_price = info.get('currentPrice')
            
            # Filter: Must be better than 2.0 (Lower is better)
            if rec_mean is None or rec_mean > 2.0: continue
            
            upside = 0
            if target_price and current_price:
                upside = round(((target_price - current_price) / current_price) * 100, 2)
            
            # Merge with existing data
            base_row = working_df[working_df['Ticker'] == ticker].iloc[0].to_dict()
            base_row['Analyst_Rating'] = rec_mean
            base_row['Target_Price'] = target_price
            base_row['Upside_%'] = upside
            
            analyst_data.append(base_row)
            time.sleep(0.2) # Polite delay
            
        except Exception:
            continue
            
    return pd.DataFrame(analyst_data)

# ==========================================
# 2. EXECUTE IT (Run this part!)
# ==========================================

# Check if fortress_df exists from the previous step
if 'fortress_df' in locals() and not fortress_df.empty:
    
    # Run the function
    Analyst_Fortress_DF = get_analyst_fortress_from_var(fortress_df)
    
    if not Analyst_Fortress_DF.empty:
        # Sort by best Analyst Rating (Lower is better) or Upside
        Analyst_Fortress_DF = Analyst_Fortress_DF.sort_values(by='Upside_%', ascending=False)
        
        # Display Results
        print("\n✅ Analyst Scan Complete!")
        print(f"Found {len(Analyst_Fortress_DF)} stocks with Buy Ratings (Score < 2.0)")
        
        # Save to CSV
        Analyst_Fortress_DF.to_csv(ANALYST_CSV, index=False)
        print("Saved to 'Analyst_Fortress_Picks.csv'")
        
        # Show top picks
        cols = ['Ticker', 'Price', 'Analyst_Rating', 'Upside_%', 'Target_Price', 'Tier']
        print(Analyst_Fortress_DF[cols].head(20))
    else:
        print("No stocks passed the Analyst filter.")
else:
    print("❌ 'fortress_df' not found or empty. Please run the Main Filter (Step 1-3) first.")


--- STEP 4: Fetching Analyst Ratings for 348 Stocks (From Memory) ---
    (Fetching serially to avoid throttling...)
 -> Analyst Scan 1/348: WPM...
 -> Analyst Scan 11/348: ANET...
 -> Analyst Scan 21/348: GRMN...
 -> Analyst Scan 31/348: CTAS...
 -> Analyst Scan 41/348: NFLX...
 -> Analyst Scan 51/348: MSFT...
 -> Analyst Scan 61/348: INTU...
 -> Analyst Scan 71/348: DV...
 -> Analyst Scan 81/348: INCY...
 -> Analyst Scan 91/348: TGTX...
 -> Analyst Scan 101/348: KEYS...
 -> Analyst Scan 111/348: WDC...
 -> Analyst Scan 121/348: PNR...
 -> Analyst Scan 131/348: HALO...
 -> Analyst Scan 141/348: CAT...
 -> Analyst Scan 151/348: AS...
 -> Analyst Scan 161/348: GLW...
 -> Analyst Scan 171/348: AMKR...
 -> Analyst Scan 181/348: EOG...
 -> Analyst Scan 191/348: PPC...
 -> Analyst Scan 201/348: IR...
 -> Analyst Scan 211/348: TTI...
 -> Analyst Scan 221/348: PAYX...
 -> Analyst Scan 231/348: VIPS...
 -> Analyst Scan 241/348: COP...
 -> Analyst Scan 251/348: EQNR...
 -> Analyst Scan 261/348

In [8]:
import pandas as pd
from yahooquery import Ticker

# ==========================================
# BUFFETT "BELOW NAV" SCAN
# ==========================================
def get_buffett_value_picks(df_input):
    print(f"\n--- STEP 5: Warren Buffett 'Below NAV' Scan ---")
    print(f"    Scanning {len(df_input)} candidates for Deep Value...")
    print("    Criteria: P/B < 1.0 (Below Book) | ROE > 0% (Profitable) | Debt/Eq < 100%")

    tickers = df_input['Ticker'].tolist()
    buffett_candidates = []

    # Use YahooQuery for speed (Key Stats are summary data, no throttling risk here)
    chunk_size = 250
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]

    for chunk in chunks:
        try:
            yq = Ticker(chunk, asynchronous=True)
            # We need defaultKeyStatistics (P/B) and financialData (ROE, Debt)
            data = yq.get_modules("defaultKeyStatistics financialData")

            for symbol in chunk:
                if isinstance(data, dict) and symbol in data:
                    try:
                        stats = data[symbol].get('defaultKeyStatistics', {})
                        fin = data[symbol].get('financialData', {})

                        # 1. Price to Book < 1.0 (The Core Rule)
                        pb = stats.get('priceToBook')
                        # Skip if None, > 1.0, or negative (insolvent)
                        if pb is None or pb >= 1.0 or pb <= 0: continue

                        # 2. Positive ROE (No Zombies)
                        roe = fin.get('returnOnEquity', 0)
                        if roe is None or roe <= 0: continue

                        # 3. Reasonable Debt (Safety)
                        # Buffett hates high leverage on weak companies
                        de = fin.get('debtToEquity', 0)
                        if de is None or de > 100: continue 

                        # Get base data from input_df
                        base_row = df_input[df_input['Ticker'] == symbol].iloc[0].to_dict()

                        # Add new Value Metrics
                        base_row['P/B Ratio'] = round(pb, 2)
                        base_row['ROE %'] = round(roe * 100, 2)
                        base_row['Debt/Eq %'] = round(de, 2)

                        buffett_candidates.append(base_row)

                    except: continue
        except: continue

    return pd.DataFrame(buffett_candidates)

# ==========================================
# EXECUTION BLOCK
# ==========================================
# Ensure we have the 'final_results' from the Main Filter
if 'final_results' in locals() and not final_results.empty:
    
    Buffett_Value_DF = get_buffett_value_picks(final_results)
    
    if not Buffett_Value_DF.empty:
        # Sort by P/B Ratio (Cheapest first)
        Buffett_Value_DF = Buffett_Value_DF.sort_values(by='P/B Ratio', ascending=True)
        
        # Save results
        Buffett_Value_DF.to_csv(BUFFETT_CSV, index=False)
        
        print("\n" + "="*60)
        print("BUFFETT SCAN COMPLETE")
        print("="*60)
        print(f"Found {len(Buffett_Value_DF)} Deep Value Stocks (Trading < Book Value)")
        print("Saved to: 'Buffett_Value_Picks.csv'")
        
        # Display
        pd.set_option('display.max_rows', 500)
        cols = ['Ticker', 'Price', 'P/B Ratio', 'ROE %', 'Debt/Eq %', 'Sector', 'Tier']
        print("\n--- DEEP VALUE PICKS ---")
        print(Buffett_Value_DF[cols].head(20))
        
    else:
        print("\n❌ No stocks passed the Buffett Value filter (All stocks are trading > Book Value).")
else:
    print("❌ 'final_results' variable not found. Please run the Main Filter first.")


--- STEP 5: Warren Buffett 'Below NAV' Scan ---
    Scanning 553 candidates for Deep Value...
    Criteria: P/B < 1.0 (Below Book) | ROE > 0% (Profitable) | Debt/Eq < 100%

BUFFETT SCAN COMPLETE
Found 19 Deep Value Stocks (Trading < Book Value)
Saved to: 'Buffett_Value_Picks.csv'

--- DEEP VALUE PICKS ---
   Ticker   Price  P/B Ratio  ROE %  Debt/Eq %                  Sector      Tier
0    GMAB   32.52       0.35  29.41       2.47              Healthcare  Fortress
14     KT   19.06       0.38   5.08      60.17  Communication Services     Risky
10   ACHC   14.50       0.42   3.69      74.49              Healthcare  Fortress
8     SSL    6.44       0.45   5.07      76.56         Basic Materials  Fortress
13   ANGI   12.77       0.56   3.42      54.03  Communication Services     Risky
7     HLX    6.28       0.59   2.71      39.52                  Energy  Fortress
4      WB   10.15       0.63  12.37      47.28  Communication Services  Fortress
16     MT   45.54       0.63   4.72      26.

In [14]:
import pandas as pd
from yahooquery import Ticker
from IPython.display import display, Markdown
import warnings
# Ignore these specific FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)
# ==========================================
# INSIDER FILTER FUNCTION
# ==========================================
def filter_for_insider_buying(tickers):
    print(f"🕵️ Scanning {len(tickers)} stocks for Insider Buying...")
    insider_picks = []
    
    # Chunk to prevent timeouts
    chunk_size = 20
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]
    
    for chunk in chunks:
        try:
            yq = Ticker(chunk, asynchronous=True)
            df_insiders = yq.insider_transactions
            
            if isinstance(df_insiders, dict) or not hasattr(df_insiders, 'reset_index'): 
                continue
            
            df_insiders = df_insiders.reset_index()

            for symbol in chunk:
                if symbol not in df_insiders['symbol'].values:
                    continue
                
                stock_tx = df_insiders[df_insiders['symbol'] == symbol].copy()
                
                # Filter for Purchases
                buys = stock_tx[stock_tx['transactionText'].astype(str).str.contains("Purchase", case=False, na=False)]
                sells = stock_tx[stock_tx['transactionText'].astype(str).str.contains("Sale", case=False, na=False)]
                
                # Logic: Net Positive Buying
                buy_vol = buys['shares'].sum() if not buys.empty else 0
                sell_vol = sells['shares'].sum() if not sells.empty else 0
                
                if buy_vol > sell_vol:
                    insider_picks.append({
                        'Ticker': symbol,
                        'Insider_Buys_Count': len(buys),
                        'Net_Shares_Bought': buy_vol - sell_vol
                    })
                        
        except Exception:
            continue

    return pd.DataFrame(insider_picks)

# ==========================================
# 2. CREATE 'Fortress_insiders' DATAFRAME
# ==========================================

# Use fortress_df if it exists, otherwise use the top 20 backup list
if 'fortress_df' in locals() and not fortress_df.empty:
    target_tickers = fortress_df['Ticker'].tolist()
else:
    # Backup list from your logs so this runs even if you restarted the kernel
    target_tickers = [
        'PET.TO', 'MFI.TO', 'TXG.TO', 'SAP.TO', 'PAAS.TO', 'NEO.TO', 'WPM.TO', 
        'FNV.TO', 'LUG.TO', 'DPM.TO', 'ASM.TO', 'PNG.V', 'DSG.TO', 'KNT.TO', 
        'GGD.TO', 'GRGD.TO', 'WDO.TO', 'OGC.TO', 'DNG.TO', 'CLS.TO'
    ]

# Run the filter and assign to the variable you requested
Fortress_insiders = filter_for_insider_buying(target_tickers)

# Display so Data Wrangler picks it up
print(f"✅ Created 'Fortress_insiders' with {len(Fortress_insiders)} rows.")
display(Fortress_insiders)

🕵️ Scanning 348 stocks for Insider Buying...
✅ Created 'Fortress_insiders' with 31 rows.


,Ticker,Insider_Buys_Count,Net_Shares_Bought
0,PGNY,3,68670.0
1,PAAS,15,1368070.0
2,VLTO,6,14327.0
3,HXL,110,1575676.0
4,SVM,3,8335.0
5,NUE,92,5442652.0
6,MGY,1,2000.0
7,MAS,10,3517712.0
8,SOLS,4,916900.0
9,OPCH,8,18954.0


In [16]:

# ==========================================
# 2. ANALYST FILTER FUNCTION FOR INSIDER PICKS (NEW)
# ==========================================
def filter_for_analyst_ratings(df_insiders, max_score=2.5):
    """
    Fetches analyst data for the insider winners and filters for 'Buy' or better.
    Scale: 1.0 = Strong Buy, 5.0 = Sell.
    Cutoff: 2.5 ensures we get 'Buy' and 'Strong Buy'.
    """
    if df_insiders.empty:
        return df_insiders
        
    tickers = df_insiders['Ticker'].tolist()
    
    
    try:
        yq = Ticker(tickers, asynchronous=True)
        # 'financial_data' contains the specific recommendation scores
        fin_data = yq.financial_data
        
        analyst_data = []
        for t in tickers:
            # Check if we got valid data for this ticker
            if isinstance(fin_data, dict) and t in fin_data:
                data = fin_data[t]
                # Ensure it's a dictionary and has the key we need
                if isinstance(data, dict) and 'recommendationMean' in data:
                    score = data.get('recommendationMean')
                    
                    # Only keep valid scores (sometimes they are None)
                    if score is not None:
                        analyst_data.append({
                            'Ticker': t,
                            'Analyst_Score': score,
                            'Analyst_Verdict': data.get('recommendationKey', 'N/A')
                        })
        
        df_analyst = pd.DataFrame(analyst_data)
        
        if df_analyst.empty:
            print("⚠️ No Analyst ratings found for these tickers.")
            return df_insiders # Return original if no data found
            
        # Merge with the Insider DataFrame
        merged = pd.merge(df_insiders, df_analyst, on='Ticker', how='inner')
        
        # FILTER: Keep only scores <= max_score (Lower is better)
        final_df = merged[merged['Analyst_Score'] <= max_score].copy()
        
        print(f"✅ Analyst Filter: {len(merged)} -> {len(final_df)} stocks (Min Rating: Buy).")
        return final_df.sort_values(by='Analyst_Score', ascending=True)

    except Exception as e:
        print(f"❌ Error in Analyst Filter: {e}")
        return df_insiders

# ==========================================
# 3. EXECUTION PIPELINE
# ==========================================

# A. Setup Tickers
if 'fortress_df' in locals() and not fortress_df.empty:
    target_tickers = fortress_df['Ticker'].tolist()
else:
    # Backup list just in case
    target_tickers = [
        'PET.TO', 'MFI.TO', 'TXG.TO', 'SAP.TO', 'PAAS.TO', 'NEO.TO', 'WPM.TO', 
        'FNV.TO', 'LUG.TO', 'DPM.TO', 'ASM.TO', 'PNG.V', 'DSG.TO', 'KNT.TO', 
        'GGD.TO', 'GRGD.TO', 'WDO.TO', 'OGC.TO', 'DNG.TO', 'CLS.TO'
    ]

# B. Run Insider Filter
insider_winners = filter_for_insider_buying(target_tickers)

# C. Run Analyst Filter (NEW STEP)
# We overwrite 'Fortress_insiders' so it works with your Data Wrangler flow
if not insider_winners.empty:
    Fortress_insiders_Analyst_buy = filter_for_analyst_ratings(insider_winners, max_score=2.5)
else:
    Fortress_insiders_Analyst_buy = pd.DataFrame()

# D. Display Result
if not Fortress_insiders_Analyst_buy.empty:
    print(f"\n🚀 Final List: {len(Fortress_insiders_Analyst_buy)} stocks (Fortress + Insider Buying + Analyst Buy Rating)")
    display(Fortress_insiders_Analyst_buy)
else:
    print("No stocks passed all filters.")

🕵️ Scanning 348 stocks for Insider Buying...
✅ Analyst Filter: 24 -> 22 stocks (Min Rating: Buy).

🚀 Final List: 22 stocks (Fortress + Insider Buying + Analyst Buy Rating)


,Ticker,Insider_Buys_Count,Net_Shares_Bought,Analyst_Score,Analyst_Verdict
4,SVM,3,8335.0,1.33333,strong_buy
22,DAR,11,703246.0,1.38462,strong_buy
5,NUE,92,5442652.0,1.46667,strong_buy
9,OPCH,8,18954.0,1.50000,strong_buy
19,CVE,73,53965480.0,1.52941,buy
12,PDD,5,16257.0,1.75610,buy
10,ALC,3,147507.0,1.78571,buy
0,PGNY,3,68670.0,1.81818,buy
1,PAAS,15,1368070.0,2.00000,buy
2,VLTO,6,14327.0,2.05263,buy


In [10]:
# ==========================================
# Watchlist Combiner (Finviz + YFinance)
# ==========================================



import pandas as pd
import yfinance as yf
from finvizfinance.quote import finvizfinance
import time
import numpy as np

# --- 1. INPUT YOUR MANUAL LIST HERE ---
MY_TICKERS = ['GRND','ARCC','BANC','ONB','UBER','ADMA','MIR','APG','SEI','FLEX','DD',] 

def get_combined_watchlist(ticker_list):
    print(f"--- Processing {len(ticker_list)} stocks ---")
    
    # --- PART A: Get Analyst Ratings from Finviz ---
    print("1. Fetching Analyst Ratings from Finviz...")
    finviz_data = []
    
    for ticker in ticker_list:
        try:
            stock = finvizfinance(ticker)
            info = stock.ticker_fundament()
            
            finviz_data.append({
                'Ticker': ticker,
                'Recom': info.get('Recom', np.nan),
                'Target_Price': info.get('Target Price', np.nan)
            })
            time.sleep(0.5) 
            
        except Exception as e:
            print(f"   Skipping Finviz for {ticker}: {e}")
            finviz_data.append({'Ticker': ticker, 'Recom': np.nan, 'Target_Price': np.nan})

    df_finviz = pd.DataFrame(finviz_data)
    
    # --- PART B: Get Real-Time Stats from yfinance ---
    print("2. Fetching Price & Volatility from yfinance...")
    
    try:
        # Download data (1 Year is perfect for 52-Week MA)
        data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)
        yf_stats = []
        
        for ticker in ticker_list:
            try:
                # --- FIXED: Robust Data Extraction ---
                if isinstance(data.columns, pd.MultiIndex):
                    if ticker in data.columns.levels[0]:
                        df = data[ticker].copy()
                    else:
                        print(f"   Warning: {ticker} not found in yfinance download.")
                        continue
                else:
                    df = data.copy()

                # Cleanup
                df = df.dropna(subset=['Close'])
                if len(df) < 20: 
                    print(f"   Warning: Not enough data for {ticker}")
                    continue

                # --- MATH CALCULATIONS ---
                current_price = df['Close'].iloc[-1]
                prev_close = df['Close'].iloc[-2]
                
                high_52 = df['High'].max()
                drop_from_high = ((current_price - high_52) / high_52) * 100
                
                change_pct = ((current_price - prev_close) / prev_close) * 100
                
                # Volatility (30-day Std Dev)
                volatility = df['Close'].pct_change().std() * 100
                
                # Relative Volume
                curr_vol = df['Volume'].iloc[-1]
                avg_vol = df['Volume'].tail(30).mean()
                rel_vol = curr_vol / avg_vol if avg_vol > 0 else 0

                # --- NEW: 52-Week Moving Average ---
                # Since we fetched exactly 1 year ('1y'), the mean of the whole column is the 52W MA
                ma_52w = df['Close'].mean()

                # Distance from MA (Optional but helpful metric)
                # dist_ma = ((current_price - ma_52w) / ma_52w) * 100 

                yf_stats.append({
                    'Ticker': ticker,
                    'Price': round(current_price, 2),
                    'Change_%': round(change_pct, 2),
                    '52W_MA': round(ma_52w, 2),          # <--- Added Here
                    'Drop_from_High_%': round(drop_from_high, 2),
                    'Volatility_%': round(volatility, 2),
                    'Rel_Volume': round(rel_vol, 2)
                })
                
            except Exception as e:
                print(f"   Error calculating stats for {ticker}: {e}")
                continue
                
        df_yf = pd.DataFrame(yf_stats)
        
    except Exception as e:
        print(f"yfinance Critical Error: {e}")
        return pd.DataFrame()

    # --- PART C: Merge ---
    if not df_finviz.empty:
        if not df_yf.empty:
            master_df = pd.merge(df_finviz, df_yf, on='Ticker', how='outer')
        else:
            master_df = df_finviz
            
        # Added '52W_MA' to this list so it displays in the final table
        cols = ['Ticker', 'Price', 'Change_%', '52W_MA', 'Drop_from_High_%', 'Recom', 'Target_Price', 'Rel_Volume', 'Volatility_%']
        
        final_cols = [c for c in cols if c in master_df.columns]
        return master_df[final_cols]
    else:
        return pd.DataFrame()

# --- RUN IT ---
watchlist_df = get_combined_watchlist(MY_TICKERS)

if not watchlist_df.empty:
    if 'Drop_from_High_%' in watchlist_df.columns:
        watchlist_df['Drop_from_High_%'] = pd.to_numeric(watchlist_df['Drop_from_High_%'], errors='coerce')
        print("\n--- Final Watchlist ---")
        display(watchlist_df.sort_values(by='Drop_from_High_%', ascending=True))
    else:
        display(watchlist_df)
else:
    print("No data found.")

--- Processing 11 stocks ---
1. Fetching Analyst Ratings from Finviz...
2. Fetching Price & Volatility from yfinance...


C:\Users\James\AppData\Local\Temp\ipykernel_30396\2779024273.py:46: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)



--- Final Watchlist ---


,Ticker,Price,Change_%,52W_MA,Drop_from_High_%,Recom,Target_Price,Rel_Volume,Volatility_%
6,GRND,13.41,-0.59,17.66,-46.64,1.40,21.75,0.52,3.18
0,ADMA,19.11,-0.88,17.90,-25.56,1.00,30.00,0.39,3.26
9,SEI,44.58,-0.16,32.53,-21.85,1.17,65.45,0.38,5.90
7,MIR,23.77,0.08,19.92,-21.49,1.12,30.62,0.45,3.49
10,UBER,81.50,0.30,84.53,-20.09,1.47,112.40,0.56,2.41
5,FLEX,62.56,-1.22,48.30,-13.38,1.50,76.00,0.30,2.90
2,ARCC,20.17,-0.15,20.48,-9.91,1.27,22.64,1.47,1.38
8,ONB,22.79,-0.83,21.40,-4.55,1.85,25.92,1.09,2.14
1,APG,39.20,-0.94,31.40,-3.40,1.45,43.40,0.34,1.92
4,DD,40.89,-0.90,31.90,-2.29,1.48,47.19,0.59,2.23


In [11]:
import requests
import pandas as pd
import google as genai
import enum
from typing_extensions import TypedDict
import json
import plotly.express as px
import sys
#!"{sys.executable}" -m pip install google.genai
#!"{sys.executable}" -m pip install plotly.express

In [12]:
tickers_gemini = ['NVDA'] 
import os
from google import genai
from google.genai import types
from IPython.display import display, Markdown

In [13]:


# ==========================================
# SECURE CONFIGURATION
# ==========================================

# 1. Define the path to your key file
# If the file is in the same folder as this notebook, just use the filename.
KEY_FILE_PATH = "C:\\Users\\James\\OneDrive - McMaster University\\Gemini API Key\\gemini_key.txt"

def load_api_key(filepath):
    """
    Reads the API key from a local file to avoid hardcoding it.
    """
    try:
        with open(filepath, "r") as f:
            # .strip() removes any accidental newlines or spaces
            return f.read().strip()
    except FileNotFoundError:
        print(f"❌ Error: Could not find the file '{filepath}'")
        print("Please create a text file with your API key in it.")
        return None
    except Exception as e:
        print(f"❌ Error reading key file: {e}")
        return None

# 2. Load the key and set the environment variable
api_key = load_api_key(KEY_FILE_PATH)

if api_key:
    os.environ["GEMINI_API_KEY"] = api_key
    print("✅ API Key loaded securely.")
else:
    print("⚠️ CRITICAL: API Key not loaded. The script will fail.")

# ==========================================
# SENTIMENT ANALYSIS FUNCTION
# ==========================================
def analyze_sentiment_gemini_3(tickers_gemini, company_name=None):
    """
    Uses Gemini 3 Flash (Preview) with 'High' Thinking Level and Google Search 
    using the NEW google-genai SDK.
    """
    if not os.environ.get("GEMINI_API_KEY"):
        print("❌ Stop: No API Key found.")
        return

    print(f"\n🧠 Gemini 3 is thinking (High Reasoning Mode)... analyzing ${tickers_gemini}...")

    # Initialize Client
    client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

    config = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            include_thoughts=False, 
            thinking_level="HIGH"
        ),
        tools=[types.Tool(
            google_search=types.GoogleSearch() 
        )],
        response_modalities=["TEXT"]
    )

    prompt = f"""
    You are a Senior Equity Research Analyst using the Gemini 3 Reasoning Engine. 
    Perform a deep "Market Sentiment Analysis" on {tickers_gemini} ({company_name if company_name else 'the company'}).
    
    Step 1: SEARCH. Use Google Search to find the latest (last 30 days) news, analyst notes, and SEC filings.
    Step 2: REASON. Analyze the search results to determine the true market psychology. Look for contradictions between price action and news.
    
    Investigate these 4 Pillars:
    1. **News Virality**: Are headlines fear-mongering or euphoric? (Look for scandals, lawsuits, or product breakthroughs).
    2. **Analyst Shifts**: Are price targets moving UP or DOWN in the last week?
    3. **Institutional Flows**: Any reports of hedge funds or insiders buying/selling?
    4. **The "Whisper" Number**: What are traders saying on forums vs. official guidance?

    **OUTPUT FORMAT:**
    Produce a professional Markdown report:
    
    ## 🧠 Gemini 3 Sentiment Report: {tickers_gemini}
    **Reasoning Depth:** High
    **Sentiment Score:** [1-10]
    **Verdict:** [Buy / Hold / Sell / Speculative]
    
    ### 1. The Bull Thesis (Why it goes up)
    * ...
    
    ### 2. The Bear Thesis (Why it goes down)
    * ...
    
    ### 3. Deep Dive Analysis
    * **News Analysis**: ...
    * **Smart Money**: ...
    * **Financial Statement Analysis**: (Historic performance over last 3 years + expected performance)
    
    ### 4. Conclusion
    [Summary of whether the current price is a trap or an opportunity]
    """

    try:
        response = client.models.generate_content(
            model='gemini-3-flash-preview', # Or 'gemini-3-flash-preview'
            contents=prompt,
            config=config
        )
        display(Markdown(response.text))
        
    except Exception as e:
        print(f"❌ Error: {e}")

# ==========================================
# EXECUTION
# ==========================================
# Only run this if the key loaded successfully
if os.environ.get("GEMINI_API_KEY"):
    analyze_sentiment_gemini_3(tickers_gemini, tickers_gemini)

✅ API Key loaded securely.

🧠 Gemini 3 is thinking (High Reasoning Mode)... analyzing $['NVDA']...


## 🧠 Gemini 3 Sentiment Report: ['NVDA']
**Reasoning Depth:** High  
**Sentiment Score:** 7.8 / 10  
**Verdict:** Buy on Weakness (Opportunity)

---

### 1. The Bull Thesis (Why it goes up)
*   **The "Inference" Pivot**: The $20 billion Groq licensing deal is a massive strategic move. It signals NVIDIA's shift from being a "training" powerhouse to dominating "low-latency inference," effectively killing the thesis that competitors like Groq could disrupt its moat.
*   **Blackwell Demand**: Management confirmed "insatiable" demand for Blackwell architecture, which is effectively sold out through mid-2026.
*   **China Re-entry**: The US authorization of H200 chip exports to China (even with the 25% tax) reopens a multi-billion dollar revenue stream previously considered "lost" due to geopolitical friction.
*   **Analyst Upward Pressure**: Recent weeks have seen targets shift from the $240 range to a consensus $275, with "whisper" targets pushing toward $300-$350 by 2026.

### 2. The Bear Thesis (Why it goes down)
*   **The "Insider Exit" Signal**: Corporate insiders, including CEO Jensen Huang, sold over $1.7 billion in shares in 2025. While often dismissed as "diversification," the sheer volume and the fact that there are *zero* insider buys in the last 12 months creates a psychological ceiling.
*   **Capital Bloat**: Strategic bets like the $5 billion Intel stake and the $20 billion Groq deal are massive deployments of cash. Bears argue this "aggressive spending" hints at a desperate attempt to defend a moat that is starting to leak.
*   **Priced for Perfection**: With Q4 revenue guidance at a staggering $65B (+/- 2%), even a "minor beat" is no longer enough to move the needle. The market demands a "beat and raise" of epic proportions.

---

### 3. Deep Dive Analysis

#### **News Analysis**
Headline sentiment is a battle between **Euphoria** (product breakthroughs) and **Fear** (regulatory scrutiny). The news cycle is currently dominated by the "Groq deal" and "China exports," which act as significant positive catalysts. However, the narrative is punctuated by news of "heavy insider dumping" and potential DOJ/Antitrust inquiries. Historically, NVIDIA’s stock has "shaken off" regulatory news in favor of pure performance, but the $1.7B insider selling is a persistent drag on sentiment.

#### **Smart Money**
*   **Institutional Flows**: Large-cap managers (BlackRock, JPMorgan, UBS) are still accumulating or holding, viewing NVDA as a "utility" of the AI era.
*   **Insider Sentiment**: **Extremely Bearish**. 511 insider sales vs. 0 buys in the last 6 months. This suggests that the people running the company believe the stock is near its peak valuation for the current cycle.

#### **Financial Statement Analysis**
*   **Historic Performance (Last 3 Years)**: 
    *   **2023**: Revenue of $26.97B (Stagnant post-crypto crash).
    *   **2024**: Revenue of $60.92B (+125% YoY).
    *   **2025**: Revenue of $130.50B (+114% YoY). 
*   **Efficiency**: Net margins have exploded from ~15% in early 2023 to a consistent **53.4%** in late 2025.
*   **Projected Performance**: TTM revenue as of Oct 2025 sits at **$187.14B**. Analysts expect FY2026 to push toward $200B+ as Blackwell becomes the primary driver. The return on equity (ROE) is currently a "best-in-class" 99%.

---

### 4. Conclusion
**Verdict: Opportunity, but avoid "Chasing the Rip."**
The current price action (~$188-$190) shows a slight disconnect; the news is overwhelmingly positive regarding the "Groq Moat" and China exports, yet the price has faced resistance. This is likely a result of **year-end profit-taking** and **insider selling pressure** rather than a fundamental decay. 

The "Whisper" expectations for the upcoming Q4 (Feb 2026 report) are set at $1.52 EPS—significantly higher than the official consensus. If the stock retreats to the $165-$175 range (near its 50-day moving average), it represents a high-probability entry point. The long-term trajectory toward $275+ remains intact, provided Blackwell production yields remain stable.